# === Design ===

## 146. LRU Cache
[146. LRU Cache](https://leetcode.com/problems/lru-cache/)

## 1396. Design Underground System
[1396. Design Underground System](https://leetcode.com/problems/design-underground-system/)

In [6]:
from collections import defaultdict
a = defaultdict(lambda: [0, 0])
a['cc'] = [1,1]
print(a)

defaultdict(<function <lambda> at 0x7f849de09670>, {'cc': [1, 1]})


In [8]:
from collections import defaultdict

class UndergroundSystem:

    def __init__(self):
        # P
        self.check_in_dict = {}
        # S chose 2 = S^2
        self.trip_dict = defaultdict(lambda: [0, 0])

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        self.check_in_dict[id] = (stationName, t)
        

    def checkOut(self, id: int, stationName: str, t: int) -> None:
        # look up the id in check_in_dict and transfer the data to trip_dict
        start_station, start_time = self.check_in_dict.pop(id)
        self.trip_dict[(start_station, stationName)][0] += t - start_time
        self.trip_dict[(start_station, stationName)][1] += 1
        

    def getAverageTime(self, startStation: str, endStation: str) -> float:
        total_time, total_count = self.trip_dict[(startStation, endStation)]
        return total_time / total_count


# Your UndergroundSystem object will be instantiated and called as such:
# obj = UndergroundSystem()
# obj.checkIn(id,stationName,t)
# obj.checkOut(id,stationName,t)
# param_3 = obj.getAverageTime(startStation,endStation)

## 1656. Design an Ordered Stream
[1656. Design an Ordered Stream](https://leetcode.com/problems/design-an-ordered-stream/)

In [2]:
"""
Basically , we need to store every incoming value at the given index. And
with every incoming index, we have to check

If the current index is less than the incoming index, the we have to return
an empty list

Else , we have to return an sliced list from the incoming index to the first index
where there is no insertion till yet.

Solution:
    Initialize a list of size n with None
    Maintain the current index with self.ptr
    For every insert call, with idKey, value
    Assign the list[idKey-1] to the value # Since array is 0-index reduce 1
    Check if the current index is less than incoming index(idKey-1) and return []
    Else return sliced list from incoming index(idKey-1) till we do not encounter None.
"""

class OrderedStream:
    def __init__(self, n: int):
        self.stream = [None for _ in range(n)]
        self.ptr = 0
        
    def insert(self, idKey: int, value: str):
        idKey -= 1
        self.stream[idKey] = value
        if self.ptr < idKey:
            return []
        else:
            while self.ptr < len(self.stream) and self.stream[self.ptr] is not None:
                self.ptr += 1
            return self.stream[idKey:self.ptr]

# Your OrderedStream object will be instantiated and called as such:
# obj = OrderedStream(n)
# param_1 = obj.insert(idKey,value)

## 380. Insert Delete GetRandom O(1)
[380. Insert Delete GetRandom O(1)](https://leetcode.com/problems/insert-delete-getrandom-o1/)

In [14]:
from random import choice
a = [1,2,3]
b = choice(a)
b

3

In [16]:
from random import choice
class RandomizedSet:

    def __init__(self):
        self.rand_list = []
        self.ele_idx = {}

    def insert(self, val: int) -> bool:
        if val in self.ele_idx:
            return False
        self.ele_idx[val] = len(self.rand_list)
        self.rand_list.append(val)
        return True

    def remove(self, val: int) -> bool:
        if val not in self.ele_idx:
            return False
        
        # find the last element
        last_ele = self.rand_list[-1]
        # find the idx of delete ele
        del_idx = self.ele_idx[val]
        
        # dict
        # update the last_ele
        self.ele_idx[last_ele] = del_idx
        # delete val
        self.ele_idx.pop(val)
        
        # list
        # self.rand_list[-1], self.rand_list[del_idx] = self.rand_list[del_idx], self.rand_list[-1]
        self.rand_list[del_idx] = last_ele
        self.rand_list.pop()
        return True

    def getRandom(self) -> int:
        return choice(self.rand_list)

## 381. Insert Delete GetRandom O(1) - Duplicates allowed
[381. Insert Delete GetRandom O(1) - Duplicates allowed](https://leetcode.com/problems/insert-delete-getrandom-o1-duplicates-allowed/)

## 1206. Design Skiplist
[1206. Design Skiplist](https://leetcode.com/problems/design-skiplist/)

# === Array/String ===

## 238. Product of Array Except Self
[238. Product of Array Except Self](https://leetcode.com/problems/product-of-array-except-self/)

In [11]:
class Solution:
    def productExceptSelf(self, nums):
        """
        uodate the answer inplace
        """
        n = len(nums)
        answer = [0]*n
        
        # answer[i] contains the product of all the elements to the left
        # Note: for the element at index '0', there are no elements to the left,
        # so the answer[0] would be 1
        answer[0] = 1
        for i in range(1, n):
            # answer[i - 1] already contains the product of elements to the left of 'i - 1'
            # Simply multiplying it with nums[i - 1] would give the product of all 
            # elements to the left of index 'i'
            answer[i] = nums[i - 1] * answer[i - 1]
        
        # R contains the product of all the elements to the right
        # Note: for the element at index 'length - 1', there are no elements to the right,
        # so the R would be 1
        R = 1;
        for i in reversed(range(n)):        
            # For the index 'i', R would contain the 
            # product of all elements to the right. We update R accordingly
            answer[i] = answer[i] * R
            R *= nums[i]
        
        return answer

In [10]:
a = [1, 2, 3]
for i in reversed(range(len(a))):
    print(a[i])

3
2
1


In [9]:
# Solution 1: TC = O(n) SC = O(n)
class Solution:
    def productExceptSelf(self, nums):
        """
        res[i] = mul(nums[0: i]) * mul(nums[i + 1:])
        
          1   2 3 4
    left  1   1 2 6
    right 24 12 4 1
        """
        n = len(nums)
        left = [1 for _ in range(n)]
        for i in range(1, n):
            left[i] = left[i - 1] * nums[i - 1]
        
        right = [1 for _ in range(n)]
        for i in range(n - 2, -1, -1):
            right[i] = right[i + 1] * nums[i + 1]
        
        res = [1 for _ in range(n)]
        for i in range(n):
            res[i] = left[i] * right[i]
        return res

## 128. Longest Consecutive Sequence
[128. Longest Consecutive Sequence](https://leetcode.com/problems/longest-consecutive-sequence/)

In [9]:
class Solution:
    def longestConsecutive(self, nums) -> int:
        """
        100,4,200,1,3,2
        i
        
        1 2 3 4 5
        
        5 4 3 2 1
                j
          
        1. Brute force --> O(n^3)
        2. 2 changes --> O(n)
        """
        if not nums:
            return 0
        num_set = set(nums)
        global_max = 0
        
        for num in num_set:
            # avoid duplicate calculation
            # start at a 断崖
            if num - 1 not in num_set:
                cur_num = num
                cur_max = 1
                while cur_num + 1 in num_set:
                    cur_num += 1
                    cur_max  += 1
                global_max = max(global_max, cur_max)
        return global_max

## 3. Longest Substring Without Repeating Characters
[3. Longest Substring Without Repeating Characters](https://leetcode.com/problems/longest-substring-without-repeating-characters/)

In [10]:
class Solution:
    def lengthOfLongestSubstring(self, s: str) -> int:
        """
        abcabcbb
        i
          j
        abcdefgg
        i
              j
        
        [i, j] current longest substring witout rep chars
        max: global_max = 1
        set[a b c
        
        TC = O(n)
        SC = O(n)
        """
        n = len(s)
        if n <= 1:
            return n
        dup = set()
        l = 0
        global_max = 0
        for r in range(n):
            while s[r] in dup:
                # shrink left
                dup.remove(s[l])
                l += 1
            # satisfy
            global_max = max(global_max, r - l + 1)
            # add current
            dup.add(s[r])
        return global_max

## 1. Two Sum
[1. Two Sum](https://leetcode.com/problems/two-sum/)
- 变种 ： two sum变形，find how many pairs‍‌‍‌‌‍‌‍‍‍‌‍‍‍‌‌‌‌‌ (of 2 elements) in an array sum up to the target value，
- sorted? duplicates?

## 394. Decode String
[394. Decode String](https://leetcode.com/problems/decode-string/)

## 1790. Check if One String Swap Can Make Strings Equal
[1790. Check if One String Swap Can Make Strings Equal](https://leetcode.com/problems/check-if-one-string-swap-can-make-strings-equal/)

- 判断两个string，如果 只做一次swap ，是否可以变的一样

## 611. Valid Triangle Number
[611. Valid Triangle Number](https://www.1point3acres.com/bbs/thread-918542-1-1.html)

## 723. Candy Crush
[723. Candy Crush](https://leetcode.com/problems/candy-crush/)

- Time complexity analysis explanation: Suppose there are R rows and C Columns. Then the number of vertical repetitions of length 3 we can crush are R/3 per column * C columns. The number of horizontal repetitions of length 3 we can crush are C/3 per row * R rows. Each crushing session takes O (R * C) time so we have O((R * C * R/3 * C) + (RCC/3R)) = O(R^2C^2/3 + R^2C^2/3) = O(R^2C^2).
- Space: O(1)

In [7]:
class Solution(object):
    def candyCrush(self, board):
        return self.cursh(board) if todo else board
    
    # one candy crush
    def crush(self, board):
        R = len(board)
        C = len(board[0])
        # flag to check if we need to recursivly call candy crush functions
        todo = False

        # mark same elements in the same row
        for r in range(R):
            for c in range(C-2):
                if abs(board[r][c]) == abs(board[r][c+1]) == abs(board[r][c+2]) != 0:
                    board[r][c] = board[r][c+1] = board[r][c+2] = -abs(board[r][c])
                    todo = True

        # mark same elements in the same col
        for r in range(R-2):
            for c in range(C):
                if abs(board[r][c]) == abs(board[r+1][c]) == abs(board[r+2][c]) != 0:
                    board[r][c] = board[r+1][c] = board[r+2][c] = -abs(board[r][c])
                    todo = True

        """
        elements drop
        start at the bottom of the row
            if not candy:
                copy wr --
            if it's candy:
                skip
        """
        for c in range(C):
            wr = R-1
            # fill drop elements
            for r in range(R-1, -1, -1):
                if board[r][c] > 0:
                    board[wr][c] = board[r][c]
                    wr -= 1
            # fill top cells to 0
            for wr in range(wr, -1, -1):
                board[wr][c] = 0

## ++++  Intervals +++

## 253. Meeting Rooms II
[253. Meeting Rooms II](https://leetcode.com/problems/meeting-rooms-ii/)

In [2]:
class Solution:
    def minMeetingRooms(self, intervals) -> int: 
        """
        Minimum number of conference room --- maximium number of overlapping at the same time. 
        [[0,30], [5,10], [15,20]] 
        
        time stamp: 
        start time: +1 
        end time: -1 
        
        list: (time,number)
        0 +1; 30 -1; 5: +1; etc. 
        
        sort() 
        
        0   5    10    15    20    30 
        1   1     -1    1     -1    -1   
        
        arr = 1   2      1     2     1    0       
        
        max(arr) 
        time: n log (n) 
        space: O(n)  
        """   
        # dict time stamps
        time_stamps = []
        for start, end in intervals:
            # cover smae time stamps
            time_stamps.append((start, 1))
            time_stamps.append((end, -1))

        # sort
        time_stamps.sort()

        # calculate the accumulated sum and keep track of the global max
        global_max = 0
        cur_room = 0
        for time, val in time_stamps:
            cur_room += val
            global_max = max(global_max, cur_room)
        # return global max
        return global_max

## 759. Employee Free Time
[759. Employee Free Time](https://leetcode.com/problems/employee-free-time/)

简单来说就是 给定一个总体的time frame，然后再给你一个time interval的list 让你找出其中空出来的interval
被挂倒是不冤 第一轮我就给出了灾难级的表现 国人大哥给接雨水那题很可能是想帮我 毕竟算是刷题网很经典的题了 可惜我没把握住 上一次做这题得半年前了
我就给了一个当时自己做的解法 然后让我优化 我依稀记得可以用单调栈 但是中间很多的细节记不起来

In [4]:
# optmize by default dict (avoid same time stamps has not interval)
class Solution:
    def employeeFreeTime(self, schedule: '[[Interval]]') -> '[Interval]':
        """
        [[1,2],   [5,6]],
        [[1,  3]],
               [[4,       10]]
         2 -1 -1 1 1  -1   -1
         2  1  0 1 2  1     0
        
        accumulated sum == 0
        
        
        edge case:
                       [94,99]
        [57,87],[91,94]
        1    -1  1  -1
                         1  -1
        1    0   1   0   1   0
        """
        # time_stamps
        time_stamps = defaultdict(int)
        for cur in schedule:
            for s in cur:
                time_stamps[s.start] += 1
                time_stamps[s.end] -= 1
        # sort
        time_sort = sorted(time_stamps.keys())
        # accumulated sum
        cur_sum = 0
        res = []
        for i in range(len(time_sort) - 1):
            pre = time_sort[i]
            val = time_stamps[pre]
            cur_sum += val
            if cur_sum == 0:
                cur = time_sort[i + 1]
                res.append(Interval(pre, cur))
        return res

In [3]:
class Solution:
    def employeeFreeTime(self, schedule: '[[Interval]]') -> '[Interval]':
        """
        [[1,2],   [5,6]],
        [[1,  3]],
               [[4,       10]]
         2 -1 -1 1 1  -1   -1
         2  1  0 1 2  1     0
        
        accumulated sum == 0
        
        
        !! edge case:
                       [94,99]
        [57,87],[91,94]
        1    -1  1  -1
                         1  -1
        1    0   1   0   1   0
        """
        # time_stamps
        time_stamps = []
        for cur in schedule:
            for s in cur:
                time_stamps.append((s.start, 1))
                time_stamps.append((s.end, -1))
        # sort
        time_stamps.sort()
        # accumulated sum
        accumulated_sum = []
        cur_sum = 0
        for time, val in time_stamps:
            cur_sum += val
            accumulated_sum.append((time, cur_sum))
        # post processing
        res = []
        for i in range(len(accumulated_sum) - 1):
            pre = accumulated_sum[i]
            cur = accumulated_sum[i + 1]
            if pre[1] == 0 and pre[0] != cur[0]:
            # if pre[1] == 0:
                res.append(Interval(pre[0], cur[0]))
        return res

## 56. Merge Intervals
[56. Merge Intervals](https://leetcode.com/problems/merge-intervals/)

In [8]:
class Solution:
    def merge(self, intervals):
        """
        TC = O(nlogn)
        SC = O(n)
        
        sort by start time
        [1,3],[2,6],[8,10],[15,18]
        compare pair by pair
        case 1: cur start > previous end (not merge)
        case 2: cur start <= previous end (merge)
        """
        res = []
        n = len(intervals)
        intervals.sort()
        res.append(intervals[0])
        for i in range(1, n):
            cur = intervals[i]
            if cur[0] > res[-1][1]:
                # not merge
                res.append(cur)
            else:
                # merge
                res[-1] = [res[-1][0], max(res[-1][1], cur[1])]
        return res

## 986. Interval List Intersections
[986. Interval List Intersections](https://leetcode.com/problems/interval-list-intersections/)

In [9]:
# Solution 2: merge intervals
class Solution:
    def intervalIntersection(self, A, B):
        ans = []
        i = j = 0

        while i < len(A) and j < len(B):
            # Let's check if A[i] intersects B[j].
            # lo - the startpoint of the intersection
            # hi - the endpoint of the intersection
            lo = max(A[i][0], B[j][0])
            hi = min(A[i][1], B[j][1])
            if lo <= hi:
                ans.append([lo, hi])

            # Remove the interval with the smallest endpoint
            if A[i][1] < B[j][1]:
                i += 1
            else:
                j += 1

        return ans

In [6]:
# Solution 1: accummulated sum
# !!! edge case same time stamp one start one end
class Solution:
    def intervalIntersection(self, firstList, secondList):
        """
        [0,  2],  [5,10]
          [1,   5]
         1 1 -1 -1 1 -1
         1 2  1  0 1  0
        
        !!! edge case
        same time stamp: one is start, one is end
        """
        time_stamps = []
        for start, end in firstList:
            time_stamps.append((start, 1))
            time_stamps.append((end, -1))
        for start, end in secondList:
            time_stamps.append((start, 1))
            time_stamps.append((end, -1))
        # sort
        time_stamps.sort()
        # find the intervals with val 2
        cur_sum = 0
        res = []
        for i in range(len(time_stamps) - 1):
            pre = time_stamps[i]
            cur = time_stamps[i + 1]
            cur_sum += pre[1]
            if cur_sum == 2 or (pre[0] == cur[0] and pre[1] + cur[1] == 0):
                res.append((pre[0], cur[0]))
        return res

# === Heap / PriorityQueue ===

## 1244. Design A Leaderboard
[1244. Design A Leaderboard](https://leetcode.com/problems/design-a-leaderboard/)

In [3]:
from collections import defaultdict
import heapq
class Leaderboard:
    # SC = O(N + K) dict + heap

    def __init__(self):
        self.scores = defaultdict(int)

    # TC = O(1)
    def addScore(self, playerId: int, score: int) -> None:
        self.scores[playerId] += score
        
    # TC = K (heapify) + (N - K)logK(pop) + KlogK(add up)
    def top(self, K: int) -> int:
        minHeap = []
        for x in self.scores.values():
            heapq.heappush(minHeap, x)
            if len(minHeap) > K:
                heapq.heappop(minHeap)
        # add top k
        total = 0
        while minHeap:
            total += heapq.heappop(minHeap)
        return total
        
    # TC = O(1)
    def reset(self, playerId: int) -> None:
        self.scores[playerId] = 0

# === HashTable ===

## 1347. Minimum Number of Steps to Make Two Strings Anagram
[1347. Minimum Number of Steps to Make Two Strings Anagram](https://leetcode.com/problems/minimum-number-of-steps-to-make-two-strings-anagram/)

In [2]:
class Solution:
    def minSteps(self, s: str, t: str) -> int:
        """
        1. calculate the char an it's count
            s leetcode -> {l:1, e:3, t:1, c:1, o:1, d:1}
            t practice -> {p:1, e:1, r:1, c:2, a:1, t:1, i:1}
        2. iterate over t:
            if ch in s:
                if s_count < t_count: need to modify the dif
                if s_count > t_count: offset the dif
           if ch not in s:
                modify ch
        """
        s_dict = Counter(s)
        t_dict = Counter(t)
        steps = 0
        for ch, count in t_dict.items():
            if ch in s_dict:
                dif = count - s_dict[ch]
                steps += dif if dif > 0 else 0
            else:
                steps += count
        return steps

## 1169. Invalid Transactions
[1169. Invalid Transactions](https://leetcode.com/problems/invalid-transactions/)

In [1]:
from collections import defaultdict
"""
Clarify:
    1. duplicate output
    
Solution:
        prev_tran = dictionary --> create a dictionary to keep track of previous transaction 
        invalid_tran = set --> to store invalid transaction /set to avoid duplication 
        
        Iterate over transactions:
            -> if the amount is greater than 1000 add it to the invalid_tran
            -> if there is any other previous transaction done by similar person , check it from the prev_tran
                    -> bring all previous transaction done by similar person (iterate over the prev_tran)
                            -> check if the absolute time difference is less than 60 and the city is different 
                            -> if that is true add it to the invalid transaction 
            -> add the transaction to the prev_tran - always keep track of previous transaction 
"""
class Solution:
    def invalidTransactions(self, transactions):
        if not transactions:
            return []
        prev_tran = defaultdict(list)
        invalid_tran = set()
        for i, tran in enumerate(transactions):
            cur_info = tran.split(',')
            # get info
            name, time, amount, city = cur_info[0], int(cur_info[1]), int(cur_info[2]), cur_info[3]
            # check amount
            if amount > 1000:
                invalid_tran.add((tran, i))
            # check same person transactions
            if name in prev_tran:
                for pre, idx in prev_tran[name]:
                    prev_info = pre.split(',')
                    prev_time = int(prev_info[1])
                    prev_city = prev_info[3]
                    if abs(prev_time - time) <= 60 and prev_city != city:
                        invalid_tran.add((pre,idx))
                        invalid_tran.add((tran, i))
            # add current transaction into pre_tran
            prev_tran[name].append((tran, i))
        return [ele[0] for ele in invalid_tran]
# TC = O(n^2)
# SC = O(n)

# === Linkedlist ===

## 83. Remove Duplicates from Sorted List
[83. Remove Duplicates from Sorted List](https://leetcode.com/problems/remove-duplicates-from-sorted-list/)

In [1]:
class Solution:
    def deleteDuplicates(self, head):
        """
         [1,   1,1,2,3,3]
                  cur 
          res
        """
        if not head:
            return head
        cur = head
        while cur and cur.next:
            if cur.val == cur.next.val:
                cur.next = cur.next.next
            else:
                cur = cur.next
        return head

## 24. Swap Nodes in Pairs
[24. Swap Nodes in Pairs](https://leetcode.com/problems/swap-nodes-in-pairs/)

In [2]:
class Solution:
    """
        1 2 3 4
        newHead = head.next
        head.next.next = reverse(head.next.next)
    """
    def swapPairs(self, head):
        # base case
        if head is None or head.next is None:
            return head
        # recursive rule
        newHead = head.next
        head.next = self.swapPairs(head.next.next)
        newHead.next = head
        return newHead

## 2. Add Two Numbers
[2. Add Two Numbers](https://leetcode.com/problems/add-two-numbers/)

In [7]:
class Solution:
    def addTwoNumbers(self, l1, l2):
        """
         2 4 3 
               i
         5 6 4 8
                 j
 carry 0
         7 0 8 8
        """
        
        # Time = O(max(m, n))   m = len(l1)
        # Space = O(max(m, n))  n = len(l2)
        dummyHead = ListNode(226)
        dummy = dummyHead
        i = l1
        j = l2
        carry = 0
        while i or j or carry:
            l1_val = i.val if i else 0
            l2_val = j.val if j else 0
            # total sum
            cur_sum = l1_val + l2_val + carry
            # set cur_val and update carry
            carry = cur_sum // 10
            cur_val = cur_sum % 10
            dummy.next = ListNode(cur_val)
            dummy = dummy.next
            i = i.next if i else None
            j = j.next if j else None
        return dummyHead.next

## 445. Add Two Numbers II

[445. Add Two Numbers II](https://leetcode.com/problems/add-two-numbers-ii/)

In [6]:
class Solution:
    # TC = O(m + n)
    # SC = O(max(m, n))
    def addTwoNumbers(self, l1, l2):
        l1 = self.reverse(l1)
        l2 = self.reverse(l2)
        carry = 0
        dummyHead = ListNode(226)
        dummy = dummyHead
        while l1 or l2 or carry > 0:
            val1 = l1.val if l1 else 0
            val2 = l2.val if l2 else 0
            cur_sum = val1 + val2 + carry
            num = cur_sum % 10
            carry = cur_sum // 10
            dummy.next = ListNode(num)
            dummy = dummy.next
            
            l1 = l1.next if l1 else None
            l2 = l2.next if l2 else None
        return self.reverse(dummyHead.next)
            
        
    def reverse(self, head):
        if head is None or head.next is None:
            return head
        newHead = self.reverse(head.next)
        head.next.next = head
        head.next = None
        return newHead

## 430. Flatten a Multilevel Doubly Linked List
[430. Flatten a Multilevel Doubly Linked List](https://leetcode.com/problems/flatten-a-multilevel-doubly-linked-list/)

In [8]:
#    TC = O(n)
#    SC = O(n)
class Solution:
    def flatten(self, head: 'Optional[Node]') -> 'Optional[Node]':
        res = head
        self.helper(head)
        return res
    
    def helper(self, head):
        # base case
        if not head:
            return head
        # keep track of the previous pointer
        pre = head
        while head:
            # if current node has child, recursion
            if head.child:
                # get the tail of the sub-problem
                tail = self.helper(head.child)
                # connect child tail and next node
                tail.next = head.next
                if head.next:
                    head.next.prev = tail
                # connct child head to current node
                head.next, head.child.prev = head.child, head
                # update previous node
                pre = tail
                # set child pointer to None
                head.child = None
                # go to next node
                head = tail.next
            else:
                pre = head
                head = head.next
        return pre

In [3]:
"""
# Definition for a Node.
class Node:
    def __init__(self, val, prev, next, child):
        self.val = val
        self.prev = prev
        self.next = next
        self.child = child
"""

class Solution:
    """
    TC = O(n)
    SC = O(n)
    1. iterate over the current level
    2. if cur node has child, recursion(keep track if the head and taill of subproblem)
    3. connect sub_problem node to cuurent level node
        
    """
    def flatten(self, head: 'Optional[Node]') -> 'Optional[Node]':
        return self.helper(head)[0]
    
    def helper(self, head):
        # base case
        if not head:
            return head, head
        # current level
        cur = head
        while cur:
            if not cur.child:
                cur = cur.next
            else:
                child_head, child_tail = self.helper(cur.child)
                # record next node
                next_node = cur.next
                if cur.next:
                    child_tail.next = next_node
                    next_node.prev = child_tail
                    
                cur.next = child_head
                child_head.prev = cur
                cur.child = None
                # next loop
                cur = next_node
        # record tail of current list
        tail = head
        while tail.next:
            tail = tail.next
        return head, tail

# === Queue/Stack/Deque ===

## +++ Monotone Stack +++

## 84. Largest Rectangle in Histogram
[84. Largest Rectangle in Histogram](https://leetcode.com/problems/largest-rectangle-in-histogram/)

In [ ]:
class Solution:
    def largestRectangleArea(self, heights) -> int:
        """
        [-1,2,1,5,6,2,3,-1]
            p

        TC = O(n)
        SC = O(n)
        """
        global_max = 0
        # add -1 to head and tail of the input list
        heights.insert(0, -1)
        heights.append(-1)
        n = len(heights)
        # initial
        stack = deque([(0, heights[0])])
        p = 1
        while p < n:
            # cliff
            if stack and stack[-1][1] > heights[p]:
                idx, h = stack.pop()
                # determine the left and right border
                l = stack[-1][0] + 1
                r = p - 1
                width = r - l + 1
                cur_area = width * h
                global_max = max(global_max, cur_area)
            else:
                # append the current height
                stack.append((p, heights[p]))
                p += 1
        return global_max

## 42. Trapping Rain Water
[42. Trapping Rain Water](https://leetcode.com/problems/trapping-rain-water/)

- Solution 1: DP
- Solution 2: Decreasing stack
- Solution 3: Two pointers

In [1]:
# Solution 1: DP
class Solution:
    def trap(self, height) -> int:
        """
        Solution 1: DP
            left[]: max height seen from current position to left
                    max(left_max[i-1], height[i])
            right[]: max height seen from current position to right
            res += min(right_max[i], left_max[i]) - height[i]
        """
        n = len(height)
        left_max = [0 for _ in range(n)]
        left_max[0] = height[0]
        for i in range(1, n):
            left_max[i] = max(left_max[i-1], height[i])

        right_max = [0 for _ in range(n)]
        right_max[n - 1] = height[n - 1]
        for i in range(n - 2, -1, -1):
            right_max[i] = max(right_max[i + 1], height[i])
        
        res = 0
        for i in range(n):
            res += min(left_max[i], right_max[i]) - height[i]
        
        return res

In [2]:
# Solution 2: Stack
class Solution:
    def trap(self, height) -> int:
        """
        Solution 2: Descresing Monotone Stack
        stack store the index of the height
        accumulate the water layer by layer
        if cur_height < top_stack(might trap water)
                push to stakl
        if cur_height > top_stack(the right_boundry is fixed)
                pop the stack and calculate the water
        """
        stack = deque([0])
        res = 0
        i = 1
        while i < len(height):
            if stack and height[stack[-1]] < height[i]:
                # trap water
                cur_hei = height[stack.pop()]
                if stack:
                    # calculate the left and right bounday
                    new_height = min(height[stack[-1]], height[i]) - cur_hei 
                    cur_width = i - stack[-1] - 1
                    res += new_height * cur_width
            else:
                stack.append(i)
                i += 1
        return res

In [3]:
# Solution 3: Two pointers
class Solution:
    def trap(self, height) -> int:
        """
        Solution 3: Two pointers
        left = 0
        right = n - 1
        """
        left = 0
        right = len(height) -1
        res = 0
        left_max = 0
        right_max = 0
        while left < right:
            if height[left] < height[right]:
                # right boundry is fixed
                # check left
                if height[left] > left_max:
                    left_max = height[left]
                else:
                    # calculate the water at left
                    res += left_max - height[left]
                left += 1
            else:
                if height[right] >= right_max:
                    right_max = height[right]
                else:
                    # calculate the right
                    res += right_max - height[right]
                right -= 1
        return res

## 496. Next Greater Element I
[496. Next Greater Element I](https://leetcode.com/problems/next-greater-element-i/)

In [5]:
from collections import deque
class Solution:
    def nextGreaterElement(self, nums1, nums2):
        """
        Hashmap {element, next_greater_element}
        Decreasing Stack
        
        TC = O(n)
        SC = O(n)
        """
        stack = deque()
        greater_dict = {}
        for i in range(len(nums2)):
            while stack and nums2[i] > stack[-1]:
                greater_dict[stack.pop()] = nums2[i]
            stack.append(nums2[i])
        # no next-greater element 
        while stack:
            greater_dict[stack.pop()] = -1
            
        # iterate over nums1
        res = []
        for num in nums1:
            res.append(greater_dict[num])
        return res

## 503. Next Greater Element II
[503. Next Greater Element II](https://leetcode.com/problems/next-greater-element-ii/)

In [ ]:
class Solution:
    def nextGreaterElements(self, nums):
        n = len(nums)
        res = [-1 for _ in range(n)]
        nums += nums
        
        stack = deque()
        for i, num in enumerate(nums):
            while stack and stack[-1][1] < num:
                res[stack.pop()[0]] = num
            if i < n:
                stack.append((i, num))
        return res

## 1472. Design Browser History
[1472. Design Browser History](https://leetcode.com/problems/design-browser-history/)

In [4]:
class BrowserHistory:
    """
    two stack to keep track of the dependency of previous and next pages
    
    homepage: b
    stack previous [ a 
    stack next [ c
    
    # !!! when visiting a new page, no next url exist
    """

    def __init__(self, homepage: str):
        self.home = homepage
        self.previous = deque()
        self.next = deque()

    def visit(self, url: str) -> None:
        self.previous.append(self.home)
        self.home = url
        # !!! when visiting a new page, no next url exist
        self.next.clear()

    def back(self, steps: int) -> str:
        while self.previous and steps > 0:
            self.next.append(self.home)
            self.home = self.previous.pop()
            steps -= 1
        return self.home

    def forward(self, steps: int) -> str:
        while self.next and steps > 0:
            self.previous.append(self.home)
            self.home = self.next.pop()
            steps -= 1
        return self.home

# === Greedy ===

## 1029. Two City Scheduling
[1029. Two City Scheduling](https://leetcode.com/problems/two-city-scheduling/)

In [18]:
# TC = O(nlogn)
# SC = O(n)
class Solution:
    def twoCitySchedCost(self, costs) -> int:
        """
        diff = [B - A] (diff is larger, the cost to B is larger)
        sort diff
        put first half people to A, and second half to B
        """
        diffs = []
        for c1, c2 in costs:
            diffs.append([c2 - c1, c1, c2])
        diffs.sort()
        res = 0
        for i in range(len(diffs)):
            if i < len(diffs) // 2:
                res += diffs[i][2]
            else:
                res += diffs[i][1]
        return res

# === Tree ===

## 98. Validate Binary Search Tree
[98. Validate Binary Search Tree](https://leetcode.com/problems/validate-binary-search-tree/)
- O(1) space

## 314. Binary Tree Vertical Order Traversal
[314. Binary Tree Vertical Order Traversal](https://leetcode.com/problems/binary-tree-vertical-order-traversal/)

## 428. Serialize and Deserialize N-ary Tree
[428. Serialize and Deserialize N-ary Tree](https://leetcode.com/problems/serialize-and-deserialize-n-ary-tree/)

## === DFS ===

## 200. Number of Islands
[200. Number of Islands](https://leetcode.com/problems/number-of-islands/)

## 694. Number of Distinct Islands
[694. Number of Distinct Islands](https://leetcode.com/problems/number-of-distinct-islands/)

## 46. Permutations
[46. Permutations](https://leetcode.com/problems/permutations/)

## 47. Permutations II
[47. Permutations II](https://leetcode.com/problems/permutations-ii/)

## 1274. Number of Ships in a Rectangle
[1274. Number of Ships in a Rectangle](https://leetcode.com/problems/number-of-ships-in-a-rectangle/)

## 79. Word Search
[79. Word Search](https://leetcode.com/problems/word-search/)

## === DP ===

## 322. Coin Change
[322. Coin Change](https://leetcode.com/problems/coin-change/)

## 140. Word Break II
[140. Word Break II](https://leetcode.com/problems/word-break-ii/)

## 935. Knight Dialer
[935. Knight Dialer](https://leetcode.com/problems/knight-dialer/)

https://www.1point3acres.com/bbs/thread-916935-1-1.html
8.2 - 8.14

https://www.1point3acres.com/bbs/thread-921835-1-1.html
8.22